In [3]:
from Input_Data import Tanners_Clean_Code

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from causalinference import CausalModel

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from scipy import stats
import linearmodels




def Clean(data):
    
    new_df = pd.read_csv(data)       

    return new_df 
cwurData = "Input_Data/cwurData.csv" ##200 Nans in broad_impact column, 2200 rows/14 columns

clean_df = Clean(cwurData)

grouped_df = clean_df.groupby(['country','year']).mean().sort_values(by='country', ascending=False)
grouped_df





world_rank  national_rank  quality_of_education  \
country        year                                                    
Uruguay        2015  867.000000           66.0            367.000000   
               2014  933.000000            1.0            355.000000   
United Kingdom 2015  425.061538           33.0            262.769231   
               2014  409.546875           32.5            250.140625   
               2013   36.714286            4.0             46.285714   
...                         ...            ...                   ...   
Australia      2012   93.000000            1.5            101.000000   
               2015  519.518519           14.0            306.222222   
               2013   94.000000            1.5            101.000000   
Argentina      2015  681.333333            2.0            286.666667   
               2014  748.500000            2.5            314.250000   

                     alumni_employment  quality_of_faculty  publications  \
country        year                                                        
Uruguay        2015         567.000000          218.000000    814.000000   
               2014         478.000000          210.000000    940.000000   
United Kingdom 2015         352.523077          182.630769    440.353846   
               2014         313.093750          178.812500    434.765625   
               2013          74.714286           45.857143     27.571429   
...                                ...                 ...           ...   
Australia      2012         101.000000          101.000000     61.500000   
               2015         417.407407          204.407407    435.962963   
               2013         101.000000           72.000000     81.000000   
Argentina      2015         526.666667          205.333333    513.000000   
               2014         477.250000          201.500000    625.750000   

                      influence   citations  broad_impact     patents  \
country        year                                                     
Uruguay        2015  669.000000  812.000000    809.000000  871.000000   
               2014  744.000000  800.000000    889.000000  737.000000   
United Kingdom 2015  387.323077  350.892308    410.784615  478.584615   
               2014  380.234375  351.984375    396.046875  449.812500   
               2013   28.571429   32.428571           NaN   68.285714   
...                         ...         ...           ...         ...   
Australia      2012   91.000000   87.000000           NaN   57.500000   
               2015  420.703704  459.111111    492.148148  508.222222   
               2013  101.000000  101.000000           NaN   74.500000   
Argentina      2015  568.333333  492.333333    642.000000  871.000000   
               2014  625.000000  544.500000    714.250000  737.000000   

                         score  
country        year             
Uruguay        2015  44.160000  
               2014  44.350000  
United Kingdom 2015  48.004615  
               2014  48.445937  
               2013  62.841429  
...                        ...  
Australia      2012  44.155000  
               2015  45.812222  
               2013  44.635000  
Argentina      2015  44.593333  
               2014  44.732500  

[152 rows x 11 columns]